This module loads the data from data.py, creates a TensorFlow/Keras model from model.py, trains the model on the data, and then saves the best model.

In [4]:
import tensorflow as tf # conda install -c anaconda tensorflow
import time
import os
import settings    # Use the custom settings.py file for default parameters

from model import load_model, get_callbacks, evaluate_model
from data import load_data

from argparser import args

if args.keras_api:
    import keras as K
else:
    from tensorflow import keras as K


/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
usage: ipykernel_launcher.py [-h] [--data_path DATA_PATH]
                             [--data_filename DATA_FILENAME]
                             [--output_path OUTPUT_PATH]
                             [--inference_filename INFERENCE_FILENAME]
                             [--use_upsampling] [--num_threads NUM_THREADS]
                             [--num_inter_threads NUM_INTER_THREADS]
                             [--batch_size BATCH_SIZE] [--blocktime BLOCKTIME]
                             [--epochs EPOCHS] [--learningrate LEARNINGRATE]
                             [--weight_dice_loss WEIGHT_DICE_LOSS]
                             [--featuremaps FEATUREMAPS] [--keras_api

SystemExit: 2

/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


For best CPU speed set the number of intra and inter threads to take advantage of multi-core systems.
See https://github.com/intel/mkl-dnn

In [ ]:
config = tf.ConfigProto(intra_op_parallelism_threads=args.num_threads,
                        inter_op_parallelism_threads=args.num_inter_threads)

sess = tf.Session(config=config)


def train_and_predict(data_path, data_filename, batch_size, n_epoch):
    """
    Create a model, load the data, and train it.
    """

    """
    Step 1: Load the data
    """
    hdf5_filename = os.path.join(data_path, data_filename)
    print("-" * 30)
    print("Loading the data from HDF5 file ...")
    print("-" * 30)

    imgs_train, msks_train, imgs_validation, msks_validation = \
        load_data(hdf5_filename)

    print("-" * 30)
    print("Creating and compiling model ...")
    print("-" * 30)

    """
    Step 2: Define the model
    """
    model = load_model(imgs_train.shape, msks_train.shape)

    model_filename, model_callbacks = get_callbacks()

    """
    Step 3: Train the model on the data
    """
    print("-" * 30)
    print("Fitting model with training data ...")
    print("-" * 30)

    history = model.fit(imgs_train, msks_train,
                        batch_size=batch_size,
                        epochs=n_epoch,
                        validation_data=(imgs_validation, msks_validation),
                        verbose=1, shuffle="batch",
                        callbacks=model_callbacks)

    # Append training log
    # with open("training.log","a+") as fp:
    #     fp.write("{}: {}\n".format(datetime.datetime.now(),
    #                              history.history["val_dice_coef"]))


    """
    Step 4: Evaluate the best model
    """
    print("-" * 30)
    print("Loading the best trained model ...")
    print("-" * 30)

    model = evaluate_model(model_filename, imgs_validation, msks_validation)

Get some details on the processor of our system

In [3]:
!lscpu

Architecture:          x86_64
CPU op-mode(s):        32-bit, 64-bit
Byte Order:            Little Endian
CPU(s):                16
On-line CPU(s) list:   0-15
Thread(s) per core:    2
Core(s) per socket:    8
Socket(s):             1
NUMA node(s):          1
Vendor ID:             GenuineIntel
CPU family:            6
Model:                 85
Model name:            Intel(R) Xeon(R) Platinum 8124M CPU @ 3.00GHz
Stepping:              4
CPU MHz:               3404.926
BogoMIPS:              6000.00
Hypervisor vendor:     KVM
Virtualization type:   full
L1d cache:             32K
L1i cache:             32K
L2 cache:              1024K
L3 cache:              25344K
NUMA node0 CPU(s):     0-15
Flags:                 fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid aperfmperf pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt tsc_deadlin

In [ ]:
import datetime
print("Started script on {}".format(datetime.datetime.now()))

print("args = {}".format(args))
os.system("uname -a")
print("TensorFlow version: {}".format(tf.__version__))
start_time = time.time()

train_and_predict(args.data_path, args.data_filename,
                      args.batch_size, args.epochs)

print(
    "Total time elapsed for program = {} seconds".format(
        time.time() -
        start_time))
print("Stopped script on {}".format(datetime.datetime.now()))

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
   http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

SPDX-License-Identifier: EPL-2.0
